In [3]:
import pandas as pd
import numpy as np

# The Case Study



Analyzing an affiliated credit card (special card only available to members of a specific group). Think of it like this the credit card company is like Citibank and the affiliated group is a company like Indian Oil or Barista. The affiliated group has to pay some fees to credit card company (because the credit card company is providing this service to the group).
Product Details :
- average card balance is Rs 1000
- each card has an interest rate of 15%
- membership fee is Rs 20 per card
- loss rate is 3% (NOTE: what are the interpretations of this ? state your assumption CLEARLY when solving )
- Rs 25 operating cost per card
- Rs 10 affiliation fee per card (cost to the group organization itself, so that their members get this awesome card program)
- 6.5% cost of funds (that credit card company must pay to the bank where it gets money)
- bill/statement is issued on 1st of every month
- credit-free period = 15 days from the bill/statement issue date.


In [4]:
# Initialization of the given parameters

total_customer=1000 # 1000 Customers

#Assuming 95% customers paid on time of credit card bill
customer_with_no_delay=95/100

# Assuming 2% paid 30 days after due date
customer_with_30_days=2/100 

# Assuming 2% paid 45 days after due date
customer_with_45_days=2/100 


# Assuming 1% paid 60 days after due date
customer_with_60_days=1/100

#Monthly average balance to be maintained
month_bal=1000

# Source of Revenue per year

mem_fee=12*20*total_customer           # membership fee is Rs 20 per card

affil_fee=12*10*total_customer         # Rs 10 per month affiliation fee per card

int_rate=15/100                        # Interest rate

# Expenses per year

cost_of_funds=12*total_customer*1000*0.065/12     # Bank lending money to the credit card company


operating_cost=12*25*1000               # Rs 25 operating cost per card


loss_rate=3/100                         # 3% loss due to non-payment of credit card bills

In [5]:
cost_of_funds

65000.0

Provide a monthly calculation of profits/losses given :

i) The interest rate is simple interest.

ii) Interest is calculated on a daily basis.

iii) The average card balance is the average end of month balance.

iv) All members are more than a year old.



Consider 1000 customers for the above scenario, divide these 1000 customers into following groups(upon your assumption and understanding about the credit lending industry)

1.customers who pay the bill within the credit-free period

2.customers paying 30 days after the last due date

3.customer paying 45 days after the last due date

4.customer paying 60 days after the last due date(remove these customers for the complete calculation of the next month,basically, you are not supposed to provide further credit services to these customers)




In [6]:
# Category Table based on default date

no_of_days=30
categories=4
category_type=[]

for customer_category in range(categories):
    
    if customer_category==0:
        category_type.append('Paid before due date')
        
    else:
        category_type.append('Paid %d days after due date'%no_of_days)
        no_of_days+=15     

Data_Calculations=pd.DataFrame({'Category':category_type},index=np.arange(1,categories+1))

Data_Calculations

,Category
1,Paid before due date
2,Paid 30 days after due date
3,Paid 45 days after due date
4,Paid 60 days after due date


In [7]:
category_type[0]

'Paid before due date'

In [8]:
category_type[1]

'Paid 30 days after due date'

In [9]:
category_type[2]

'Paid 45 days after due date'

In [10]:
category_type[3]

'Paid 60 days after due date'

In [11]:
#Interest after 'N' days = N*outstanding balance*Interest rate per year/365

def interest_calc(payment_days):
    
    # Billing cycle starts from 1st of every month
    average_day_trans= 15  # Assuming average day of transaction as 15th of billing cycle 
    
    if payment_days==category_type[0]:
        Int_per_card=0
        
    #Interest for 30 days
    
    elif payment_days==category_type[1]:
        
        Int_per_card=(average_day_trans+15+30)*month_bal*int_rate/365
        
    #Interest for 45 days
    elif payment_days==category_type[2]:
        Int_per_card=(average_day_trans+15+45)*month_bal*int_rate/365
    #Interest for 60 days
    else:
        Int_per_card=(average_day_trans+15+60)*month_bal*int_rate/365
        
    return Int_per_card 

In [12]:
def total_int(column):
    
    payment_days=column[0]
    interest_per_card=column[1]
    
    if payment_days==category_type[0]:
        total_interest=0
        
    #Interest for all customers: late by 30 days
    
    elif payment_days==category_type[1]:
        total_interest=12*interest_per_card*customer_with_30_days*total_customer
        
    #Interest for all customers: late by 45 days
    
    elif payment_days==category_type[2]:
        total_interest=12*interest_per_card*customer_with_45_days*total_customer
        
   #Interest for all customers: late by 60 days, remove these customers after non payment

    else:
        total_interest=12*interest_per_card*customer_with_60_days*total_customer
        
    return total_interest

In [13]:
def PnL_check(): # Column name based on profit/loss
    
    if net_profit_loss>=0:
        
        return 'Profit margin in percentage'
    
    else:
        
        return 'Loss margin in percentage'

In [14]:
Data_Calculations

,Category
1,Paid before due date
2,Paid 30 days after due date
3,Paid 45 days after due date
4,Paid 60 days after due date


In [15]:
Data_Calculations['Estimated_Interest_on_Default']=Data_Calculations['Category'].apply(interest_calc).round(2)

Data_Calculations['Total_Interest_1Year']=Data_Calculations[['Category','Estimated_Interest_on_Default']].apply(total_int,axis=1)

Data_Calculations['Fixed_Late_Fee_charge']=Data_Calculations['Estimated_Interest_on_Default'].apply(lambda x: 0 if x==0 else 50)

Data_Calculations

,Category,Estimated_Interest_on_Default,Total_Interest_1Year,Fixed_Late_Fee_charge
1,Paid before due date,0.00,0.0,0
2,Paid 30 days after due date,24.66,5918.4,50
3,Paid 45 days after due date,30.82,7396.8,50
4,Paid 60 days after due date,36.99,4438.8,50


In [16]:
Expenses=cost_of_funds+operating_cost+(loss_rate*total_customer*month_bal)

Earnings=mem_fee+affil_fee+np.sum(Data_Calculations['Total_Interest_1Year'])+(12*max(Data_Calculations['Fixed_Late_Fee_charge'])*5/100*total_customer)

margin=Earnings-Expenses # margin earned above or below total amount spent

net_profit_loss=margin/Expenses*100
    
report=pd.DataFrame({'Total yearly card balance':12*month_bal*total_customer,'Earnings':Earnings,'Expenses':Expenses,
              'Margin':margin,PnL_check():net_profit_loss.round(2)},index=[' '])
report

,Total yearly card balance,Earnings,Expenses,Margin,Profit margin in percentage
,12000000,407754.0,395000.0,12754.0,3.23
